# Step 1: Find Unusual Patterns in Hourly Google Search Traffic

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read search data into DataFrame (replace with actual file path or file variable)
search_data = pd.read_csv('/path_to_google_search_data.csv', parse_dates=True, index_col='date')

# Slice data to May 2020
may_2020_data = search_data.loc['2020-05']

# Plot May 2020 data to visualize unusual patterns
may_2020_data.plot(title="Google Search Traffic for May 2020", figsize=(10,6))
plt.show()

# Calculate total traffic for May 2020
total_traffic_may = may_2020_data['search_traffic'].sum()

# Compare May traffic to median across all months
monthly_median = search_data.resample('M').sum()['search_traffic'].median()

# Print results
print(f"Total search traffic for May 2020: {total_traffic_may}")
print(f"Monthly median search traffic: {monthly_median}")

# Analysis of traffic patterns:
if total_traffic_may > monthly_median:
    print("Google search traffic increased during the month that MercadoLibre released its financial results.")
else:
    print("Google search traffic did not increase during the month of financial results.")

# Step 2: Mine the Search Traffic Data for Seasonality

In [ ]:
# Group by hour of day
avg_hourly_traffic = search_data.groupby(search_data.index.hour).mean()['search_traffic']
avg_hourly_traffic.plot(title="Average Hourly Google Search Traffic", figsize=(10,6))
plt.show()

# Group by day of the week
avg_daily_traffic = search_data.groupby(search_data.index.dayofweek).mean()['search_traffic']
avg_daily_traffic.plot(title="Average Daily Google Search Traffic", figsize=(10,6))
plt.show()

# Group by week of the year
avg_weekly_traffic = search_data.groupby(search_data.index.week).mean()['search_traffic']
avg_weekly_traffic.plot(title="Average Weekly Google Search Traffic", figsize=(10,6))
plt.show()

# Analysis:
# Based on the plots, check if any trends exist (by hour, day of week, or week of the year).

# Step 3: Relate the Search Traffic to Stock Price Patterns

In [ ]:
# Read stock price data into DataFrame (replace with actual file path or file variable)
stock_data = pd.read_csv('/path_to_stock_price_data.csv', parse_dates=True, index_col='date')

# Concatenate stock and search data into one DataFrame
combined_data = pd.concat([search_data['search_traffic'], stock_data], axis=1)

# Slice data to the first half of 2020
first_half_2020 = combined_data.loc['2020-01':'2020-06']

# Plot search traffic and stock price for the first half of 2020
fig, ax1 = plt.subplots(figsize=(10,6))

ax1.plot(first_half_2020.index, first_half_2020['search_traffic'], color='tab:blue', label='Search Traffic')
ax2 = ax1.twinx()
ax2.plot(first_half_2020.index, first_half_2020['stock_price'], color='tab:red', label='Stock Price')

ax1.set_xlabel('Date')
ax1.set_ylabel('Search Traffic', color='tab:blue')
ax2.set_ylabel('Stock Price', color='tab:red')

plt.title('Google Search Traffic and Stock Price (Jan to Jun 2020)')
plt.show()

# Create Lagged Search Trends
combined_data['Lagged_Search_Traffic'] = combined_data['search_traffic'].shift(1)

# Create Stock Volatility (4-hour exponentially weighted rolling average)
combined_data['Stock_Volatility'] = combined_data['stock_price'].ewm(span=4).std()

# Create Hourly Stock Return (percent change)
combined_data['Hourly_Stock_Return'] = combined_data['stock_price'].pct_change()

# Display head of combined data
combined_data.head()

# Step 4: Create a Time Series Model with Prophet

In [ ]:
from fbprophet import Prophet

# Prepare data for Prophet
prophet_data = search_data.reset_index()[['date', 'search_traffic']].rename(columns={'date': 'ds', 'search_traffic': 'y'})

# Initialize and fit the Prophet model
model = Prophet()
model.fit(prophet_data)

# Create a future dataframe
future = model.make_future_dataframe(periods=365, freq='H')

# Predict future search traffic
forecast = model.predict(future)

# Plot the forecast
model.plot(forecast)
plt.title('Prophet Forecast of Google Search Traffic')
plt.show()

# Plot the individual components of the model
model.plot_components(forecast)
plt.show()

# Analysis:
# Answer questions regarding time of day, day of the week, and lowest point of search traffic.